In [8]:
import os
import numpy as np
import librosa
import tensorflow as tf

In [9]:
def extract_features(file_path, max_pad_len=174):
    try:
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        if audio.size == 0:
            return None
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        if pad_width > 0:
            mfccs = np.pad(mfccs, pad_width=((0,0),(0,pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :max_pad_len]
        return mfccs
    except Exception as e:
        print(f"❌ Error with {file_path}: {e}")
        return None

In [10]:
# Path to model and dataset
MODEL_PATH = r"D:\language classifier\Indian-Language-Classifier\language_classifier.h5"
DATA_PATH  = r"D:\language classifier\Indian-Language-Classifier\data"

# Load the trained model
model = tf.keras.models.load_model(MODEL_PATH)

# Class labels (from directory names)
classes = os.listdir(DATA_PATH)
print("✅ Model and classes loaded!")
print("Languages:", classes)


✅ Model and classes loaded!
Languages: ['Bengali', 'Gujarati', 'Hindi', 'Kannada', 'Malayalam', 'Marathi', 'Punjabi', 'Tamil', 'Telugu', 'Urdu']


In [11]:
def predict_language(file_path):
    if os.path.exists(file_path):
        mfccs = extract_features(file_path)
        if mfccs is not None:
            mfccs = np.expand_dims(mfccs, axis=(0, -1))  # reshape for model
            prediction = np.argmax(model.predict(mfccs), axis=1)[0]
            return classes[prediction]
        else:
            return "❌ Could not extract features"
    else:
        return "❌ File not found"


In [97]:
# Example test file
test_file = r"D:\language classifier\Indian-Language-Classifier\data\Urdu\21959.mp3"

predicted_lang = predict_language(test_file)
print(f"🔮 Predicted Language for {test_file}: {predicted_lang}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
🔮 Predicted Language for D:\language classifier\Indian-Language-Classifier\data\Urdu\21959.mp3: Urdu
